# Example Code
So let's start with some code. We start of course with a Hello World example. We'll make a client and a server. The client sends "Hello" to the server, which replies with "World"
![Request-Reply](https://github.com/imatix/zguide/raw/master/images/fig2.png)

In [1]:
# The server
import zmq
import time

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")
while True:
    # Wait for next request from client
    message = socket.recv()
    print("Received request: %s" % message)
    
    # Do some work
    time.sleep(1)
    
    # Send reply back to client
    socket.send(b"World")


In [ ]:
# The client
import zmq

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server...")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")

#  Do 10 requests, waiting each time for a response
for request in range(10):
    print("Sending request %s ..." % request)
    socket.send(b"Hello")

    #  Get the reply.
    message = socket.recv()
    print("Received reply %s [ %s ]" % (request, message))


Lets query the ZeroMQ version

In [4]:
print("Current libzmq version is %s" % zmq.zmq_version())
print("Current pyzmq version is %s" % zmq.__version__)

Current libzmq version is 4.1.6
Current pyzmq version is 17.0.0


# One-way data distribution (Publish/Subscribe)
In this patter a server pushes updates to a set of clients.
![Publish subscribe architecture](https://github.com/imatix/zguide/raw/master/images/fig4.png)

In [ ]:

#
#   Weather update server
#   Binds PUB socket to tcp://*:5556
#   Publishes random weather updates
#

import zmq
from random import randrange

context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://*:5556")

while True:
    zipcode = randrange(1, 100000)
    temperature = randrange(-80, 135)
    relhumidity = randrange(10, 60)

    socket.send_string("%i %i %i" % (zipcode, temperature, relhumidity))


In [ ]:

#
#   Weather update client
#   Connects SUB socket to tcp://localhost:5556
#   Collects weather updates and finds avg temp in zipcode
#

import sys
import zmq

#  Socket to talk to server
context = zmq.Context()
socket = context.socket(zmq.SUB)

print("Collecting updates from weather server…")
socket.connect("tcp://localhost:5556")

# Subscribe to zipcode, default is NYC, 10001
zip_filter = sys.argv[1] if len(sys.argv) > 1 else "10001"

# Python 2 - ascii bytes to unicode str
if isinstance(zip_filter, bytes):
    zip_filter = zip_filter.decode('ascii')
socket.setsockopt_string(zmq.SUBSCRIBE, zip_filter)

# Process 5 updates
total_temp = 0
for update_nbr in range(5):
    string = socket.recv_string()
    zipcode, temperature, relhumidity = string.split()
    total_temp += int(temperature)

print("Average temperature for zipcode '%s' was %dF" % (
      zip_filter, total_temp / (update_nbr+1))
)


# Multi socket poller :
This example show how we can read from multiple socket

In [ ]:
# multisocket.py
import zmq

# Prepare our context and sockets
context = zmq.Context()

# Connect to task ventilator
receiver = context.socket(zmq.PULL)
receiver.connect("tcp://127.0.0.1:5555")

# Connect to weather server
subscriber = context.socket(zmq.SUB)
subscriber.connect("tcp://localhost:5556")
subscriber.setsockopt(zmq.SUBSCRIBE, b"10001")

# Initialize poll set
poller = zmq.Poller()
poller.register(receiver, zmq.POLLIN)
poller.register(subscriber, zmq.POLLIN)

# Process messages from both sockets
while True:
    try:
        socks = dict(poller.poll())
    except KeyboardInterrupt:
        break

    if receiver in socks:
        message = receiver.recv()
        # process task
        print("Puller got message %s" % message) 

    if subscriber in socks:
        message = subscriber.recv()
        zipcode, temperature, relhumidity = message.split()
        print("subscriber received data %s %s %s" % (zipcode, temperature, relhumidity))


In [ ]:
# push.py
import zmq
import time
context = zmq.Context()

socket = context.socket(zmq.PUSH)
socket.bind("tcp://127.0.0.1:5555")

for request in range(10):
    print("Sending request %s ..." % request)
    socket.send(b"Hello from pusher")
    time.sleep(1)


In [ ]:
# publisher.py
import zmq
from random import randrange

context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://*:5556")

while True:
    zipcode = randrange(1, 100000)
    temperature = randrange(-80, 135)
    relhumidity = randrange(10, 60)

    socket.send_string("%i %i %i" % (zipcode, temperature, relhumidity))


your can run those file in a separate terminal:
~~~
python multisocket.py
python push.py
python publisher.py
~~~